<div align="center">
    <h1>Activation Maximization on C3D</h1>
</div>

<div align="center">
    <h4>Keras C3D model pretrained on Sports1M dataset</h4>
</div>

In [ ]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, ZeroPadding3D
from keras.layers.core import Dense, Dropout, Flatten

model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', name='conv1', strides=(1, 1, 1), input_shape=(16, 112, 112, 3)))
model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1'))
# 2nd layer group
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same', name='conv2', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool2'))
# 3rd layer group
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3a', strides=(1, 1, 1)))
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3'))
# 4th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool4'))
# 5th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5b', strides=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5'))
model.add(Flatten())
# FC layers group
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(487, activation='softmax', name='fc8'))

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

Possible to [use the pre-trained model in Caffe format or convert it to Keras format](https://github.com/axon-research/c3d-keras) or simply download model weights in Keras format from [here](https://www.dropbox.com/s/vh293aba931wrk1/sports1M_weights_tf.h5?dl=0).

In [3]:
url = "https://www.dropbox.com/s/vh293aba931wrk1/sports1M_weights_tf.h5?dl=1"  # dl=1 is important
import urllib.request
u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('sports1M_weights_tf.h5', "wb") as f :
    f.write(data)

Let's _**load weights**_ and _**compile the model**_

In [16]:
model.load_weights('sports1M_weights_tf.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')

For further visualizing of activation we need to have _**indices of 3D convolutional layers**_

In [17]:
import keras
layer_indices = []
number_of_layers = len(model.layers)
for i in range(number_of_layers):
    if type(model.layers[i]) is keras.layers.convolutional.Conv3D:
        layer_indices.append(i)
layer_indices

[0, 2, 4, 5, 7, 8, 10, 11]

Let's get _**number of filters**_ for each convolution layer

In [18]:
from vis.visualization import get_num_filters

number_of_filters = []
for idx in layer_indices:
  number_of_filters.append(get_num_filters(model.layers[idx]))
number_of_filters

[64, 128, 256, 256, 512, 512, 512, 512]

Getting activations for _**first filter**_ (indices from 0) of layer _**conv3a**_

In [19]:
from vis.visualization import visualize_activation
conv1_0 = visualize_activation(model, layer_idx=0, filter_indices=0)

Creating **gif** to visualize 3D activation

In [20]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import ImageSequenceClip

conv1_0_clip = ImageSequenceClip(list(conv1_0), fps=10).resize(1.0)
conv1_0_clip.write_gif("conv1_0.gif", fps=10)


[MoviePy] Building file conv1_0.gif with imageio


100%|██████████| 17/17 [00:00<00:00, 147.34it/s]


<img src="conv1_0.gif" alt="" height=200 width=200></img>

<div align="center">
    <h4>Keras C3D model pretrained on Sports1M dataset</h4>
</div>

<div align="center">
    <h4>conv1</h4>
</div>

<table border="5" bordercolor="blue" align="center">
		<tr>
			<td><img src="../data/layer0/layer0_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
 </table>

<div align="center">
    <h4>conv2</h4>
</div>

<table>
        <tr>
			<td><img src="../data/layer2/layer2_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv3a</h4>
</div>

<table>
		<tr>
			<td><img src="../data/layer4/layer4_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv3b</h4>
</div>

<table>
        <tr>
			<td><img src="../data/layer5/layer5_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv4a</h4>
</div>

<table>
		<tr>
			<td><img src="../data/layer7/layer7_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv4b</h4>
</div>

<table>
        <tr>
			<td><img src="../data/layer8/layer8_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv5a</h4>
</div>

<table>
    	<tr>
			<td><img src="../data/layer10/layer10_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv5b</h4>
</div>

<table>
    	<tr>
			<td><img src="../data/layer11/layer11_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter3.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>